# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [54]:
TRAINING_DATA_FILE = "SpamData/02_Training/train-data.txt"
TEST_DATA_FILE = "SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE = "SpamData/03_Testing/prob-spam.txt"
TOKEN_HAM_PROB_FILE = "SpamData/03_Testing/prob-nonspam.txt"
TOKEN_ALL_PROB_FILE = "SpamData/03_Testing/prob-all-tokens.txt"

TEST_FEATURE_MATRIX = "SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE = "SpamData/03_Testing/test-target.txt"

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=" ", dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=" ", dtype=int)

In [4]:
print("Nr of rows in training file", sparse_train_data.shape[0])
print("Nr of rows in test file", sparse_test_data.shape[0])

Nr of rows in training file 258360
Nr of rows in test file 117776


In [5]:
print("Nr of emails in training file", np.unique(sparse_train_data[:, 0]).size)
print("Nr of emails in test file", np.unique(sparse_test_data[:, 0]).size)

Nr of emails in training file 4013
Nr of emails in test file 1724


### How to create an Empty DataFrame

In [6]:
column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [7]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [8]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [9]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe
    
    Keyword argumets:
    sparse matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column.
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column.
    cat_idx -- position of the label (spam is 1, nonspam is 0) in the sparse matrix. Default: 3rd column.
    freq_idx -- position of occurrenece of word in the sparse matrix. Default: 4th column.
    """
    
    column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(0, nr_words))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(0, sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, "DOC_ID"] = doc_nr
        full_matrix.at[doc_nr, "CATEGORY"] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
    full_matrix.set_index("DOC_ID", inplace=True)
    
    return full_matrix

In [10]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: total: 14.2 s
Wall time: 14.3 s


In [11]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [25]:
nr_spam = full_train_data[full_train_data.CATEGORY == 1].CATEGORY.shape[0]
prob_spam = nr_spam / full_train_data.shape[0]
prob_spam

0.310989284824321

## Total Number of Words / Tokens

In [26]:
full_train_features = full_train_data.loc[:, full_train_data.columns != "CATEGORY"]
email_lengths = full_train_features.sum(axis=1)   #Tokens in each doc
email_lengths.shape

(4013,)

In [27]:
total_wc = email_lengths.sum()
total_wc

429241

## Number of Tokens in Spam & Ham Emails

In [33]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_wc = spam_lengths.sum()
spam_wc

176334

In [34]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_wc = ham_lengths.sum()
ham_wc

252907

In [36]:
print("Average nr of words in spam emails {:.0f}".format(spam_wc / spam_lengths.shape[0]))
print("Average nr of words in ham emails {:.0f}".format(ham_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 141
Average nr of words in ham emails 91


## Summing the Tokens Occurring in Spam

In [45]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # Adding 1 to make sure each token (word) occurs at least once (LaPlace Smoothing)

In [46]:
summed_spam_tokens.tail()

2495     3
2496     2
2497    17
2498    15
2499     2
dtype: int64

## Summing the Tokens Occurring in Ham

In [42]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [43]:
summed_ham_tokens.tail()

2495    27
2496    35
2497    20
2498    15
2499    28
dtype: int64

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [47]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE) # Add Vocab_Size because we added 1 to every token
prob_tokens_spam

0       0.012184
1       0.005228
2       0.006805
3       0.011307
4       0.006816
          ...   
2495    0.000017
2496    0.000011
2497    0.000095
2498    0.000084
2499    0.000011
Length: 2500, dtype: float64

In [48]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [49]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE) # Add Vocab_Size because we added 1 to every token
prob_tokens_ham

0       0.021472
1       0.010141
2       0.008007
3       0.003673
4       0.006311
          ...   
2495    0.000106
2496    0.000137
2497    0.000078
2498    0.000059
2499    0.000110
Length: 2500, dtype: float64

In [50]:
prob_tokens_ham.sum()

1.0

# P(Token) - Probability that Token Occurs

In [51]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [53]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [56]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

CPU times: total: 6.52 s
Wall time: 6.66 s


In [57]:
full_test_features = full_test_data.loc[:, full_test_data.columns != "CATEGORY"]
full_test_target = full_test_data.CATEGORY

In [58]:
np.savetxt(TEST_FEATURE_MATRIX, full_test_features)
np.savetxt(TEST_TARGET_FILE, full_test_target)